<a href="https://colab.research.google.com/github/samikshamudunuri/DL-Assignment-2/blob/main/Transliteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
file_path = "/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.train.tsv"
with open(file_path, 'r', encoding='utf-8') as f:
    print(f.readline())  # Prints first line of the file


अँग्री	angry	3



In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
file_path = "/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.train.tsv"
input_texts, target_texts = [], []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        eng, mar = line.strip().split('\t')[:2]
        input_texts.append(eng)
        target_texts.append('\t' + mar + '\n')  # start and end token




In [11]:
file_path = "/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.train.tsv"
input_texts, target_texts = [], []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        eng, mar = line.strip().split('\t')[:2]
        input_texts.append(eng)
        target_texts.append('\t' + mar + '\n')  # start and end token

In [12]:
input_chars = sorted(set("".join(input_texts)))
target_chars = sorted(set("".join(target_texts)))

input_token_index = {char: i+1 for i, char in enumerate(input_chars)}  # 0 = padding
target_token_index = {char: i+1 for i, char in enumerate(target_chars)}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

input_vocab_size = len(input_token_index) + 1
target_vocab_size = len(target_token_index) + 1


In [13]:
encoder_input_data = [[input_token_index[c] for c in seq] for seq in input_texts]
decoder_input_data = [[target_token_index[c] for c in seq] for seq in target_texts]
decoder_target_data = [seq[1:] for seq in decoder_input_data]  # shifted by 1

max_encoder_seq_length = max(len(seq) for seq in encoder_input_data)
max_decoder_seq_length = max(len(seq) for seq in decoder_input_data)

encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_decoder_seq_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_decoder_seq_length, padding='post')

decoder_target_data_onehot = tf.keras.utils.to_categorical(decoder_target_data, num_classes=target_vocab_size)

In [17]:
def build_seq2seq_model(cell_type='LSTM', embedding_dim=64, hidden_dim=128, num_layers=1):
    RNN = {'RNN': SimpleRNN, 'LSTM': LSTM, 'GRU': GRU}[cell_type]

    encoder_inputs = Input(shape=(None,))
    x = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)

    states = []
    for i in range(num_layers):
        return_seq = (i != num_layers - 1)
        if cell_type == 'LSTM':
            x, state_h, state_c = RNN(hidden_dim, return_sequences=return_seq, return_state=True)(x)
            states = [state_h, state_c]
        else:
            x, state_h = RNN(hidden_dim, return_sequences=return_seq, return_state=True)(x)
            states = [state_h]

    decoder_inputs = Input(shape=(None,))
    y = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)

    if cell_type == 'LSTM':
        y, _, _ = RNN(hidden_dim, return_sequences=True, return_state=True)(y, initial_state=states)
    else:
        y, _ = RNN(hidden_dim, return_sequences=True, return_state=True)(y, initial_state=states)

    decoder_outputs = Dense(target_vocab_size, activation='softmax')(y)

    return Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
model = build_seq2seq_model(cell_type='LSTM', embedding_dim=64, hidden_dim=128, num_layers=1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

X_train, X_test, y_train, y_test, dy_train, dy_test = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data_onehot, test_size=0.1, random_state=42)

model.fit([X_train, y_train], dy_train,
          batch_size=64,
          epochs=30,
          validation_split=0.1)


Epoch 1/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7216 - loss: 1.0717 - val_accuracy: 0.7765 - val_loss: 0.7413
Epoch 2/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.7904 - loss: 0.6935 - val_accuracy: 0.8410 - val_loss: 0.5189
Epoch 3/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.8629 - loss: 0.4543 - val_accuracy: 0.9031 - val_loss: 0.3202
Epoch 4/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9115 - loss: 0.2955 - val_accuracy: 0.9239 - val_loss: 0.2519
Epoch 5/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9297 - loss: 0.2350 - val_accuracy: 0.9355 - val_loss: 0.2152
Epoch 6/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9389 - loss: 0.2024 - val_accuracy: 0.9394 - val_loss: 0.1971
Epoch 7/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9443 - loss: 0.1822 - val_accuracy: 0.9452 - val_loss: 0.1796
Epoch 8/30
713/713 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9479 - loss: 0.1686 - val_accurac

In [18]:
def build_seq2seq_model(cell_type='LSTM', embedding_dim=64, hidden_dim=128, num_layers=1):
    RNN = {'RNN': SimpleRNN, 'LSTM': LSTM, 'GRU': GRU}[cell_type]

    encoder_inputs = Input(shape=(None,))
    enc_embed = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)

    if cell_type == 'LSTM':
        encoder_outputs, state_h, state_c = RNN(hidden_dim, return_state=True)(enc_embed)
        encoder_states = [state_h, state_c]
    else:
        encoder_outputs, state_h = RNN(hidden_dim, return_state=True)(enc_embed)
        encoder_states = [state_h]

    decoder_inputs = Input(shape=(None,))
    dec_embed = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)

    if cell_type == 'LSTM':
        decoder_outputs, _, _ = RNN(hidden_dim, return_sequences=True, return_state=True)(dec_embed, initial_state=encoder_states)
    else:
        decoder_outputs, _ = RNN(hidden_dim, return_sequences=True, return_state=True)(dec_embed, initial_state=encoder_states)

    decoder_dense = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    return model, encoder_inputs, encoder_states, decoder_inputs, dec_embed


In [19]:
model, encoder_inputs, encoder_states, decoder_inputs, dec_embed = build_seq2seq_model()

# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference model
latent_dim = encoder_states[0].shape[-1]

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Pass through the same LSTM layer again (reuse)
decoder_lstm = model.layers[-3]  # Assuming this is your LSTM layer
decoder_outputs, state_h, state_c = decoder_lstm(dec_embed, initial_state=decoder_states_inputs)

decoder_outputs = model.layers[-1](decoder_outputs)  # dense layer
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs, state_h, state_c]
)


In [21]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    decoded_sentence = ""

    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Fix here
        sampled_token_index = np.argmax(output_tokens[0])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            break

        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()



In [22]:
# Let's evaluate the first few test samples
print("Sample Predictions:\n")

for i in range(5):
    input_seq = encoder_input_data[i:i+1]
    decoded_sentence = decode_sequence(input_seq)

    print(f"Input    : {input_texts[i]}")
    print(f"Predicted: {decoded_sentence}")
    print(f"Target   : {target_texts[i][1:-1]}")
    print("-" * 30)


Sample Predictions:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3